# Nondeterministic Actions: Solving Tic-Tac-Toe with And-Or-Tree Search

## Introduction 
 
Multiplayer games can be implemented as:
1. __Nondeterministic actions:__ The opponent is seen as part of an environment with nondeterministic actions. Non-determinism is the result of the unknown opponent's moves. 
2. Adversarial search (the opponent acts strategic): Minimax search with optimal decisions.__
3. Adversarial search: Monte Carlo Tree search (simulate playouts). 

Here we will implement search for Tic-Tac-Toe (see [rules](https://en.wikipedia.org/wiki/Tic-tac-toe)).

We will implement
* __And-Or-Tree search.__

Each action consists of the move by the player (Or levels in the tree) and all possible (i.e., nondeterministic) responses by the opponents (and levels). The action therefore results in a set of possible states called a __belief state.__

We will search for a __conditional plan__ using And-Or-Tree search. 

## State Space and Search Tree Size

Each state is a possible board. Each of the 9 squares can have 3 values (empty, x and o), but some boards are impossible (where a player has several sequences of 3).The number of states in the state space graph is less than:

In [1]:
3**9

19683

A search tree can be superimposed on the state space graph. Note that a state can be in several branches of the tree resulting in more notes. We collapse the 

* The complete search tree has a maximal depth $m=9$
* The max branching factor $b=9$ (for first move).

DFS has a time complecity of $O(b^m)$ and a space complexity of $O(bm)$.

The number of terminal nodes of the complete search tree are less than:

In [2]:
import math

math.factorial(9)

362880

We can reach a complete board with that many sequences. Some sequences are cut short because of a win and therefore there are less terminal notes.

__Note:__ This size makes this a very small problem that can be easily solved by searching the complete game tree. Most games and real problems are to large and can only be.

## The board

I represent the board as a vector of length 9. The values are `' ', 'x', 'o'`.  

In [3]:
def empty_board():
    return [' '] * 9

board = empty_board()
display(board)

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

Some helper functions.

In [4]:
import numpy as np

def show_board(board):
    """display the board"""
    b = np.array(board).reshape((3,3))
    print(b)

board = empty_board()
show_board(board)    

print()
print("Add some x's")
board[0] = 'x'; board[3] = 'x'; board[6] = 'x';  
show_board(board)

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

Add some x's
[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]


In [5]:
def check_win(board):
    """check the board and return one of x, o, d (draw), or n (for next move)"""
    # check rows and columns
    board = np.array(board).reshape((3,3))
    
    for a_board in [board, np.transpose(board)]:
        for row in a_board:
            if len(set(row)) == 1 and row[0] != ' ':
                return row[0]
    
    # check diagonal
    if len(set([board[i][i] for i in range(len(board))])) == 1 and board[0][0] != ' ':
        return board[0][0]
    if len(set([board[i][len(board)-i-1] for i in range(len(board))])) == 1 and board[0][len(board)-1] != ' ':
        return board[0][len(board)-1]

    # check for draw
    if(np.sum(board == ' ') < 1):
        return 'd'
    
    return 'n'

show_board(board)
print('Win? ' + check_win(board))

print()
show_board(empty_board())
print('Win? ' + check_win(empty_board()))

[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]
Win? x

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
Win? n


In [6]:
def get_actions(board):
    """return possible actions as a vector ot indices"""
    return np.where(np.array(board) == ' ')[0].tolist()

show_board(board)
get_actions(board)

[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]


[1, 2, 4, 5, 7, 8]

## Recursive DFS Algorithm for And-Or-Tree Search

See AIMA page 125. 

For simplicity we implementation the search for player 'x' and we search for a conditional plan as a list of lists.

Modifications to the algorithm in the textbook:
* Modify the `results()` function to return for a state $\times$ action combination a belief state that reflects all possible responses by the opponent.
* Removed path since it is not used.
* No cycle checking (not needed).
* Goal: 
    - End search (prune subtree) when player loses. 
    - Check for loss also in the "and" phase.
    - Draw can be set as a goal state or a loss. Since DSF finds the first solution, it might be a draw while there is still a solution where the player could win. Considering draw a loss will prune the "draw" solutions and leave only the "wins," if any.


In [7]:
def results(state, action, player = 'x', other = 'o'):
    """produce the belief state after the provided action for player. 
       The belief state is the set of boards with the action and all possible reactions by the opponent."""
    state = state.copy()
    state[action] = player
    
    r = list()
    o_actions = get_actions(state)
    
    if len(o_actions) < 1 : return [state]
    
    for o_a in o_actions:
        s = state.copy()
        s[o_a] = other
        r.append(s)    
    
    return r

show_board(empty_board())

print()
print("Belief state for placing an x at position 4 of an empty board:")
results(empty_board(), 4)

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

Belief state for placing an x at position 4 of an empty board:


[['o', ' ', ' ', ' ', 'x', ' ', ' ', ' ', ' '],
 [' ', 'o', ' ', ' ', 'x', ' ', ' ', ' ', ' '],
 [' ', ' ', 'o', ' ', 'x', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', 'o', 'x', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', 'x', 'o', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', 'x', ' ', 'o', ' ', ' '],
 [' ', ' ', ' ', ' ', 'x', ' ', ' ', 'o', ' '],
 [' ', ' ', ' ', ' ', 'x', ' ', ' ', ' ', 'o']]

In [8]:
def is_terminal(state, draw = True):
    goal = check_win(state)        
    if goal == 'x': return 'win' 
    if goal == 'd': 
        if draw: return 'draw' 
        else: return None 
    if goal == 'o': return None  # loss is failure
    return False # continue

print(is_terminal(['x'] * 9))
print(is_terminal(['o'] * 9))
print(is_terminal(empty_board()))

win
None
False


In [9]:
# define global variables
DEBUG = 0
COUNT = 0

def and_or_search(board, draw = True):
    """start the search. Consider draw a goal state?"""
    global DEBUG, COUNT
    COUNT = 0
    
    plan = or_search(board, draw)
    
    if DEBUG >= 1: print(f"Number of nodes searched: {COUNT}")  
    
    return plan

def or_search(state, draw):
    """Or step of the search: try all possible action and returns a conditional plan for the first action 
    that only has goal states as leaf nodes. If none can be found then failure (None) is returned."""
    global DEBUG, COUNT
    COUNT += 1
    
    # goal check: I guess I can only lose or have draw here! Win is in during and search.
    g = is_terminal(state, draw)
    if g != False: return(g)
        
    #if is_cycle(path) return None  # no cycles for this problem
    
    # check all possible actions in the state
    for action in get_actions(state):
        plan = and_search(results(state, action), draw)
        if plan is not None: return [action, plan]
    return None

def and_search(states, draw):
    """And step of the search: follow all possible states (call the or step). 
    Return a conditional plan only if all paths lead to a goal state."""
    global DEBUG, COUNT
    COUNT += 1
    
    # return plans if no state fails
    plans = []
    for s in states:    
        # added another goal check after my move. I think I cannot have a loss here.
        g = is_terminal(s, draw)
        if g != False: return(g)
      
        plan = or_search(s, draw)
        if plan is None: return None
        plans.append(['if', s, 'then', plan]) # + else?

    return plans

# Some Tests

In [10]:
DEBUG = 1

In [11]:
# x is about to win

board = empty_board() 
board[0] = 'x'
board[1] = 'o'
board[3] = 'o'
board[4] = 'x'

print("Board:")
show_board(board)

print()
print("Win or draw:")
display(and_or_search(board))

Board:
[['x' 'o' ' ']
 ['o' 'x' ' ']
 [' ' ' ' ' ']]

Win or draw:
Number of nodes searched: 22


[2,
 [['if', ['x', 'o', 'x', 'o', 'x', 'o', ' ', ' ', ' '], 'then', [6, 'win']],
  ['if',
   ['x', 'o', 'x', 'o', 'x', ' ', 'o', ' ', ' '],
   'then',
   [5,
    [['if', ['x', 'o', 'x', 'o', 'x', 'x', 'o', 'o', ' '], 'then', [8, 'win']],
     ['if',
      ['x', 'o', 'x', 'o', 'x', 'x', 'o', ' ', 'o'],
      'then',
      [7, 'draw']]]]],
  ['if',
   ['x', 'o', 'x', 'o', 'x', ' ', ' ', 'o', ' '],
   'then',
   [5,
    [['if', ['x', 'o', 'x', 'o', 'x', 'x', 'o', 'o', ' '], 'then', [8, 'win']],
     ['if',
      ['x', 'o', 'x', 'o', 'x', 'x', ' ', 'o', 'o'],
      'then',
      [6, 'win']]]]],
  ['if',
   ['x', 'o', 'x', 'o', 'x', ' ', ' ', ' ', 'o'],
   'then',
   [5,
    [['if',
      ['x', 'o', 'x', 'o', 'x', 'x', 'o', ' ', 'o'],
      'then',
      [7, 'draw']],
     ['if',
      ['x', 'o', 'x', 'o', 'x', 'x', ' ', 'o', 'o'],
      'then',
      [6, 'win']]]]]]]

In [12]:
print("Win only:")
display(and_or_search(board, draw = False))

Win only:
Number of nodes searched: 27


[2,
 [['if', ['x', 'o', 'x', 'o', 'x', 'o', ' ', ' ', ' '], 'then', [6, 'win']],
  ['if', ['x', 'o', 'x', 'o', 'x', ' ', 'o', ' ', ' '], 'then', [8, 'win']],
  ['if',
   ['x', 'o', 'x', 'o', 'x', ' ', ' ', 'o', ' '],
   'then',
   [5,
    [['if', ['x', 'o', 'x', 'o', 'x', 'x', 'o', 'o', ' '], 'then', [8, 'win']],
     ['if',
      ['x', 'o', 'x', 'o', 'x', 'x', ' ', 'o', 'o'],
      'then',
      [6, 'win']]]]],
  ['if', ['x', 'o', 'x', 'o', 'x', ' ', ' ', ' ', 'o'], 'then', [6, 'win']]]]

In [13]:
# x can draw if it chooses 7.

board = empty_board() 
board[0] = 'x'
board[1] = 'o'
board[2] = 'x'
#board[3] = 'o'
board[4] = 'o'

print("Board:")
show_board(board)

print()
print("Win or draw:")
display(and_or_search(board))

Board:
[['x' 'o' 'x']
 [' ' 'o' ' ']
 [' ' ' ' ' ']]

Win or draw:
Number of nodes searched: 56


[7,
 [['if',
   ['x', 'o', 'x', 'o', 'o', ' ', ' ', 'x', ' '],
   'then',
   [5,
    [['if', ['x', 'o', 'x', 'o', 'o', 'x', 'o', 'x', ' '], 'then', [8, 'win']],
     ['if',
      ['x', 'o', 'x', 'o', 'o', 'x', ' ', 'x', 'o'],
      'then',
      [6, 'draw']]]]],
  ['if',
   ['x', 'o', 'x', ' ', 'o', 'o', ' ', 'x', ' '],
   'then',
   [3,
    [['if',
      ['x', 'o', 'x', 'x', 'o', 'o', 'o', 'x', ' '],
      'then',
      [8, 'draw']],
     ['if',
      ['x', 'o', 'x', 'x', 'o', 'o', ' ', 'x', 'o'],
      'then',
      [6, 'win']]]]],
  ['if',
   ['x', 'o', 'x', ' ', 'o', ' ', 'o', 'x', ' '],
   'then',
   [3,
    [['if',
      ['x', 'o', 'x', 'x', 'o', 'o', 'o', 'x', ' '],
      'then',
      [8, 'draw']],
     ['if',
      ['x', 'o', 'x', 'x', 'o', ' ', 'o', 'x', 'o'],
      'then',
      [5, 'draw']]]]],
  ['if',
   ['x', 'o', 'x', ' ', 'o', ' ', ' ', 'x', 'o'],
   'then',
   [3,
    [['if', ['x', 'o', 'x', 'x', 'o', 'o', ' ', 'x', 'o'], 'then', [6, 'win']],
     ['if',
      ['x', '

In [14]:
print("Win only:")
display(and_or_search(board, draw = False))

Win only:
Number of nodes searched: 52


None

In [15]:
# o is about to win

board = empty_board() 
board[0] = 'o'
board[1] = 'o'
board[3] = 'o'
board[4] = 'x'
board[8] = 'x'

print("Board:")
show_board(board)

print()
print("Win or draw:")
display(and_or_search(board))

Board:
[['o' 'o' ' ']
 ['o' 'x' ' ']
 [' ' ' ' 'x']]

Win or draw:
Number of nodes searched: 7


None

In [16]:
print("Win only:")
display(and_or_search(board, draw = False))

Win only:
Number of nodes searched: 7


None

In [17]:
# Empty board: Only a draw an be guaranteed

board = empty_board() 

print("Board:")
show_board(board)

print()
print("Win or draw:")
display(and_or_search(board))

Board:
[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

Win or draw:
Number of nodes searched: 833


[0,
 [['if',
   ['x', 'o', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
   'then',
   [2,
    [['if',
      ['x', 'o', 'x', 'o', ' ', ' ', ' ', ' ', ' '],
      'then',
      [4,
       [['if',
         ['x', 'o', 'x', 'o', 'x', 'o', ' ', ' ', ' '],
         'then',
         [6, 'win']],
        ['if',
         ['x', 'o', 'x', 'o', 'x', ' ', 'o', ' ', ' '],
         'then',
         [5,
          [['if',
            ['x', 'o', 'x', 'o', 'x', 'x', 'o', 'o', ' '],
            'then',
            [8, 'win']],
           ['if',
            ['x', 'o', 'x', 'o', 'x', 'x', 'o', ' ', 'o'],
            'then',
            [7, 'draw']]]]],
        ['if',
         ['x', 'o', 'x', 'o', 'x', ' ', ' ', 'o', ' '],
         'then',
         [5,
          [['if',
            ['x', 'o', 'x', 'o', 'x', 'x', 'o', 'o', ' '],
            'then',
            [8, 'win']],
           ['if',
            ['x', 'o', 'x', 'o', 'x', 'x', ' ', 'o', 'o'],
            'then',
            [6, 'win']]]]],
        ['if',
        

In [18]:
print("Win only:")
display(and_or_search(board, draw = False))

Win only:
Number of nodes searched: 13023


None

In [19]:
# o went first

board = empty_board() 
board[4] = 'o'


print("Board:")
show_board(board)

print()
print("Win or draw:")
display(and_or_search(board))

Board:
[[' ' ' ' ' ']
 [' ' 'o' ' ']
 [' ' ' ' ' ']]

Win or draw:
Number of nodes searched: 370


[0,
 [['if',
   ['x', 'o', ' ', ' ', 'o', ' ', ' ', ' ', ' '],
   'then',
   [7,
    [['if',
      ['x', 'o', 'o', ' ', 'o', ' ', ' ', 'x', ' '],
      'then',
      [6,
       [['if',
         ['x', 'o', 'o', 'o', 'o', ' ', 'x', 'x', ' '],
         'then',
         [5, 'draw']],
        ['if',
         ['x', 'o', 'o', ' ', 'o', 'o', 'x', 'x', ' '],
         'then',
         [3, 'win']],
        ['if',
         ['x', 'o', 'o', ' ', 'o', ' ', 'x', 'x', 'o'],
         'then',
         [3, 'win']]]]],
     ['if',
      ['x', 'o', ' ', 'o', 'o', ' ', ' ', 'x', ' '],
      'then',
      [5,
       [['if',
         ['x', 'o', 'o', 'o', 'o', 'x', ' ', 'x', ' '],
         'then',
         [6, 'draw']],
        ['if',
         ['x', 'o', ' ', 'o', 'o', 'x', 'o', 'x', ' '],
         'then',
         [2, 'draw']],
        ['if',
         ['x', 'o', ' ', 'o', 'o', 'x', ' ', 'x', 'o'],
         'then',
         [2, 'draw']]]]],
     ['if',
      ['x', 'o', ' ', ' ', 'o', 'o', ' ', 'x', ' '],
      

In [20]:
print("Win only:")
display(and_or_search(board, draw = False))

Win only:
Number of nodes searched: 720


None